# Data Preparation Notebook

This notebook demonstrates data preparation workflows for the Azure ML Fabric Demo predictive analytics MVP.

## Objectives
- Load and explore synthetic datasets
- Perform data quality validation
- Prepare data for machine learning models
- Create feature engineering pipelines

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Configure display options
pd.set_option('display.max_columns', None)
plt.style.use('seaborn-v0_8')
sns.set_palette('husl')

## 1. Load Synthetic Datasets

In [ ]:
# Load manufacturing data
with open('../data/synthetic/manufacturing_data.json', 'r') as f:
    manufacturing_data = json.load(f)
    
manufacturing_df = pd.DataFrame(manufacturing_data)
manufacturing_df['timestamp'] = pd.to_datetime(manufacturing_df['timestamp'])

print(f"Manufacturing Data Shape: {manufacturing_df.shape}")
print(f"Date Range: {manufacturing_df['timestamp'].min()} to {manufacturing_df['timestamp'].max()}")
manufacturing_df.head()

In [ ]:
# Load time-series seasonal data
with open('../data/synthetic/timeseries_seasonal.json', 'r') as f:
    timeseries_data = json.load(f)
    
timeseries_df = pd.DataFrame(timeseries_data)
timeseries_df['date'] = pd.to_datetime(timeseries_df['date'])

print(f"Time-series Data Shape: {timeseries_df.shape}")
print(f"Date Range: {timeseries_df['date'].min()} to {timeseries_df['date'].max()}")
timeseries_df.head()

In [ ]:
# Load equipment telemetry data
with open('../data/synthetic/equipment_telemetry.json', 'r') as f:
    telemetry_data = json.load(f)
    
telemetry_df = pd.DataFrame(telemetry_data)
telemetry_df['timestamp'] = pd.to_datetime(telemetry_df['timestamp'])

print(f"Telemetry Data Shape: {telemetry_df.shape}")
print(f"Unique Equipment: {telemetry_df['equipment_id'].nunique()}")
print(f"Equipment Types: {telemetry_df['equipment_id'].unique()}")
telemetry_df.head()

## 2. Data Quality Validation

In [ ]:
# Manufacturing data quality checks
print("Manufacturing Data Quality Report:")
print(f"Missing values:\n{manufacturing_df.isnull().sum()}")
print(f"\nOutput quantity stats:\n{manufacturing_df['output_quantity'].describe()}")
print(f"\nDefect rate stats:\n{manufacturing_df['defect_rate'].describe()}")
print(f"\nMachine efficiency stats:\n{manufacturing_df['machine_efficiency'].describe()}")

In [ ]:
# Time-series seasonal pattern validation
print("Time-series Seasonal Pattern Analysis:")
seasonal_range = timeseries_df['seasonal_component'].max() - timeseries_df['seasonal_component'].min()
print(f"Seasonal component range: {seasonal_range:.2f}")
print(f"Has clear seasonal pattern: {seasonal_range > 10}")

# Plot seasonal pattern
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(timeseries_df['date'][:100], timeseries_df['value'][:100])
plt.title('Time-series Values (First 100 days)')
plt.xticks(rotation=45)

plt.subplot(1, 2, 2)
plt.plot(timeseries_df['day_of_year'], timeseries_df['seasonal_component'], alpha=0.6)
plt.title('Seasonal Component by Day of Year')
plt.xlabel('Day of Year')
plt.ylabel('Seasonal Component')

plt.tight_layout()
plt.show()

## 3. Feature Engineering

In [ ]:
# Create derived features for manufacturing data
manufacturing_df['hour'] = manufacturing_df['timestamp'].dt.hour
manufacturing_df['day_of_week'] = manufacturing_df['timestamp'].dt.dayofweek
manufacturing_df['efficiency_category'] = pd.cut(
    manufacturing_df['machine_efficiency'], 
    bins=[0, 0.85, 0.95, 1.0], 
    labels=['Low', 'Medium', 'High']
)

print("Manufacturing Features Created:")
print(f"Efficiency categories: {manufacturing_df['efficiency_category'].value_counts()}")

In [ ]:
# Equipment telemetry anomaly scoring
def calculate_anomaly_score(row):
    """Simple anomaly scoring based on threshold deviations"""
    score = 0
    if row['temperature'] > 85: score += 1
    if row['vibration'] > 3: score += 1
    if row['pressure'] < 8 or row['pressure'] > 30: score += 1
    return score

telemetry_df['anomaly_score'] = telemetry_df.apply(calculate_anomaly_score, axis=1)
telemetry_df['is_anomalous'] = telemetry_df['anomaly_score'] > 0

print(f"Equipment Anomaly Detection Results:")
print(f"Anomalous readings: {telemetry_df['is_anomalous'].sum()}")
print(f"Normal readings: {(~telemetry_df['is_anomalous']).sum()}")

## 4. Data Export for ML Pipeline

In [ ]:
# Prepare datasets for machine learning
print("Preparing data for ML pipeline...")

# Save processed datasets
manufacturing_df.to_csv('../data/processed/manufacturing_processed.csv', index=False)
timeseries_df.to_csv('../data/processed/timeseries_processed.csv', index=False)
telemetry_df.to_csv('../data/processed/telemetry_processed.csv', index=False)

print("Data preparation complete!")
print(f"Processed files saved to data/processed/ directory")